# Tigergraph Queries

## Overview

Experimenting with Tigergraph queries

In [32]:
import os
import pyTigerGraph as tg
from dotenv import load_dotenv
import json
import pandas as pd

load_dotenv()

print("Import successful.")

Import successful.


In [2]:
tg_graph = 'marites'
tg_host = os.environ.get("TG_HOST")
tg_secret = os.environ.get("TG_SECRET")
tg_password = os.environ.get("TG_PASSWORD")

In [3]:
token = tg.TigerGraphConnection(host=tg_host, graphname=tg_graph).getToken(tg_secret)[0]
conn = tg.TigerGraphConnection(host=tg_host, graphname=tg_graph, password=tg_password, apiToken=token)

In [4]:
q = conn.gsql("select count(*) from user")
q

'[{\n"count": 115,\n"v_type": "user"\n}]'

In [5]:
conn.gsql('select * from user where primary_id=="elonmusk"')

'[{\n"v_id": "elonmusk",\n"attributes": {\n"name": "Elon Musk",\n"username": "elonmusk"\n},\n"v_type": "user"\n}]'

## get_following query

```
CREATE QUERY get_user_topics(VERTEX<user> u) FOR GRAPH marites {
  start = {u};
  all_topics = SELECT t 
               FROM user:s-(created_post:cp)-post:p-(topic_sentiment:ts)-topic:t 
               WHERE ts.sentiment == "NEUTRAL" OR ts.sentiment == "POSITIVE";
  PRINT all_topics;
}
```

In [25]:
print(conn.gsql('''
use graph marites

drop query get_following

create query get_following(vertex<user> p) for graph marites {
    start = {p};
    tgt = select t from start:s-(following:e)-user:t;
    print tgt;
}

install query get_following
'''))

Using graph 'marites'
Successfully dropped queries on the graph 'marites': [hello].
Successfully created queries: [hello].
Start installing queries, about 1 minute ...
hello query: curl -X GET 'https://127.0.0.1:9000/query/marites/hello?p=VALUE'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
Select 'm1' as compile server, now connecting ...
Node 'm1' is prepared as compile server.

Query installation finished.


In [31]:
q = conn.gsql('run query hello({})'.format('"elonmusk"'))
print(q)

{
"error": false,
"message": "",
"version": {
"schema": 0,
"edition": "enterprise",
"api": "v2"
},
"results": [{"tgt": [
{
"v_id": "WorldAndScience",
"attributes": {
"name": "World and Science",
"username": "WorldAndScience"
},
"v_type": "user"
},
{
"v_id": "4thFromOurStar",
"attributes": {
"name": "Mars",
"username": "4thFromOurStar"
},
"v_type": "user"
},
{
"v_id": "universal_sci",
"attributes": {
"name": "Universal-Sci",
"username": "universal_sci"
},
"v_type": "user"
},
{
"v_id": "archillect",
"attributes": {
"name": "Archillect",
"username": "archillect"
},
"v_type": "user"
},
{
"v_id": "yousuck2020",
"attributes": {
"name": "前澤友作",
"username": "yousuck2020"
},
"v_type": "user"
},
{
"v_id": "platobooktour",
"attributes": {
"name": "Rebecca Newberger Goldstein",
"username": "platobooktour"
},
"v_type": "user"
},
{
"v_id": "dogecoin",
"attributes": {
"name": "Dogecoin",
"username": "dogecoin"
},
"v_type": "user"
},
{
"v_id": "inspiration4x",
"attributes": {
"name": "Inspiration4",
"

In [4]:
print(conn.gsql('''
use graph marites

drop query get_user_topics

create query get_user_topics(vertex<user> u) for graph marites {
    start = {u};
    all_topics = select t
                 from user:s-(created_post>:cp)-post:p-(topic_sentiment>:ts)-topic:t
                 where (ts.sentiment == "POSITIVE") and (t.topic_type == "ATTRIBUTE" or t.topic_type == "ORGANIZATION");
    print all_topics;
}

install query get_user_topics
'''))

Using graph 'marites'
Semantic Check Fails: These queries could not be found anywhere: [get_user_topics].
Successfully created queries: [get_user_topics].
Start installing queries, about 1 minute ...
get_user_topics query: curl -X GET 'https://127.0.0.1:9000/query/marites/get_user_topics?u=VALUE'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
Select 'm1' as compile server, now connecting ...
Node 'm1' is prepared as compile server.

Query installation finished.


In [49]:
topic_res_str = conn.gsql('run query get_user_topics("{}")'.format('elonmusk'))

In [50]:
topic_res = json.loads(topic_res_str)

In [51]:
topics = list(map(lambda x: x['attributes'], topic_res['results'][0]['all_topics']))

In [66]:
topic_df = pd.DataFrame(topics)

In [58]:
print(topic_df.head())
print(topic_df.shape)

           text    topic_type
0  @MahowaldARK  ORGANIZATION
1    businesses  ORGANIZATION
2       company  ORGANIZATION
3       Bitcoin  ORGANIZATION
4      dogecoin  ORGANIZATION
(100, 2)


In [84]:
# Clean the data

pronouns = [
    'i',
    'we',
    'you',
    'he',
    'she',
    'it',
    'they',
    'me',
    'us',
    'him',
    'her',
    'them',
    'ours',
    'yours',
    'his',
    'hers',
    'ours',
    'theirs',
    'myself',
    'yourself',
    'himself',
    'herself',
    'ourselves',
    'itself',
    'themselves',
    'each other',
    'one another',
    'that',
    'which',
    'who',
    'whom',
    'whose',
    'where',
    'when',
    'what',
    'whatever',
    'anything',
    'anybody',
    'anyone',
    'something',
    'somebody',
    'nothing',
    'nobody',
    'none',
    'no one'
]

# Remove all symbols
topic_df.text = topic_df.text.str.replace('[^0-9a-zA-Z]', '')

# Convert everything to lowercase
topic_df.text = topic_df.text.str.lower()

# Remove pronouns
topic_df = topic_df[~topic_df.text.isin(pronouns)]

# Remove numbers
topic_df = topic_df[~topic_df.text.str.contains(r'\d')].head(50)

# Remove blank strings
topic_df = topic_df[topic_df.text.str.strip().str.len() != 0]

topic_df.head(100)

list(topic_df.text)

/var/folders/4f/zf_kr4_16jvb5njtdm05rm440000gn/T/ipykernel_11536/1826478866.py:52: FutureWarning: The default value of regex will change from True to False in a future version.
  topic_df.text = topic_df.text.str.replace('[^0-9a-zA-Z]', '')


['mahowaldark',
 'businesses',
 'company',
 'bitcoin',
 'dogecoin',
 'sls',
 'lux',
 'banks',
 'team',
 'tokens',
 'force',
 'yoodli',
 'mercury',
 'teslagigafactory',
 'marshelicopter',
 'ford',
 'microsoft',
 'waterfront',
 'design',
 'hifglobal',
 'gmail',
 'bluetooth',
 'transformer',
 'nasawebbtelescope',
 'insaitinstitute',
 'senpower',
 'pretend',
 'bitpay',
 'openai',
 'society',
 'cnn',
 'communitycrypto',
 'almere',
 'zune',
 'miri',
 'columbia',
 'vendors',
 'org',
 'booster',
 'ratio',
 'design',
 'tesla',
 'view',
 'volkswagen',
 'nat',
 'colour',
 'pricedrop',
 'jll',
 'science']